In [1]:
using Revise
using Pkg
TAMBO_PATH = "/Users/jlazar/research/TAMBO-MC/Tambo/"
Pkg.activate(TAMBO_PATH)
using Tambo
using CSV
using StatsBase
using Distributions
using JLD2
using Glob
using Plots

include("../figures/paperstyle.jl")

  Activating project at `~/research/TAMBO-MC/Tambo`


no_bg_dark! (generic function with 1 method)

In [94]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

PowerLaw(γ=2.37, emin=1000.0 GeV, emax=1.0e9 GeV)

In [95]:
es = 10 .^ (log(10, 300units.TeV):0.001:log(10, 100units.PeV))

fluxes = Φ.(es)

(sum(fluxes[2:end] .* diff(es)), sum(fluxes[1:end-1] .* diff(es)))

(5.962982617947055e-39, 5.995612320660388e-39)

In [23]:
using QuadGK

In [104]:
# really janky function for estimating the neutrino charged current
# cross section via linear interpolation
# This should not be used for anything else
function σ(E)
    xs6 = 9e-34units.cm^2
    xs10 = 2.5e-32units.cm^2
    m = (log(xs10) - log(xs6)) / (log(1e10units.GeV) - log(1e6units.GeV))
    b = log(xs6)
    return exp((log(E)-log(1e6units.GeV)) * m + b)
end

σ (generic function with 1 method)

In [105]:
proton_mass = 0.938units.GeV

9.38e8

In [106]:
σ(1e7units.GeV)

5.306326657434708e-24

In [90]:
# function to find the expected event rate per cm^2 per sr
# This assumes that the neutrino only sees rock
# Since there is also air, this will overestimate the rate
f(le) = (
    exp(le)
    * Φ(exp(le)) # flux
    * σ(exp(le)) # cross section
    * Tambo.lepton_range(exp(le), 15) # lepton range in mwe
    * 2.6 / proton_mass # convert to number of targets
)

f (generic function with 1 method)

In [91]:
i, err = quadgk(f, log(300units.TeV), log(100units.PeV))

(7.705536691852141e-43, 3.589570625082753e-53)

In [96]:
i * ((2units.km)^2 * π) * (π * 1.4539947544389964) * units.second * 10^7.5

54.564520173407956